In [ ]:
import sys
print(sys.version)

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
#pd.options.display.max_columns = None
#pd.options.display.float_format = '{:,.2f}'.format
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
csl_df = pd.read_sas('../data/csl/csllinkedbypreg.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')
csl_df.head()

In [ ]:
csl_df.info()

In [ ]:
#csl_df.applymap(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))
#csl_df['Marital'].apply(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))

In [ ]:
def convert_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':
            # This is the really inefficient part
            for val in df[col]:
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            if intcol_flag:
                print(f"{col} is integer")
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
            else:
                print(f"{col} is float")
        else:
            print(f"{col} is {df[col].dtype}")
    return df

In [ ]:
csl_df.agg(['count', 'size', 'nunique'])

In [ ]:
csl_df.describe()

In [ ]:
csl_df.agg(['min','max'])

In [ ]:
csl_df.dtypes

In [ ]:
def convert_integer (df):
    type_dict = {}
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':
            for val in df[col]:
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
        print(f"{col} is {df[col].dtype}")
        type_dict[col] = df[col].dtype
    return type_dict

In [ ]:
%%time
type_dict = convert_integer(csl_df)

In [ ]:
print(type_dict)

In [ ]:
csl_df.columns

In [ ]:
types = csl_df.dtypes
types.value_counts()

In [ ]:
types[(types != 'object') & (types != float)].index.values

In [ ]:
csl_df.isnull().sum()

In [ ]:
csl_df.info()

In [ ]:
csl_df.describe(include='all')

In [ ]:
#csl_df.to_csv('/home/mbopf/MFMDatasets/CSL_StudyItems/mbopf/csl_all.csv')

In [ ]:
# Clear some memory
#import gc
#del csl_df
#gc.collect()

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(10)
df = pd.DataFrame(np.random.choice([1, 2, 3.3, 5000, 111111, np.NaN], (3,9)), 
                  columns=[f'col{i}' for i in range(9)])


In [ ]:
convert_integer(df)

In [ ]:
df.max(axis=1)

In [ ]:
s = pd.cut(df.max(), bins=[0, 127, 32767, 2147483647], labels=['Int8', 'Int16', 'Int32'])
print(s)
s = s.where(df.dtypes=='float')  # So we only cast the previously float cols 
print(s)

#for idx, gp in s.groupby(s):
#    df.loc[:, gp.index] = df.loc[:, gp.index].astype(idx)